# Minibatch FFNN Implementation Evaluation
### Group Members 
1. 13516044 - Annisa Sekar Ayuningtyas
2. 13516078 - Muhammad Alif Arifin
3. 13516152 - Deborah Aprilia Josephine

## Import Library and Dataset

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from MiniBatch import MiniBatch

df = pd.read_csv('dataset.csv')
df

,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes
5,rainy,65,70,True,no
6,overcast,64,65,True,yes
7,sunny,72,95,False,no
8,sunny,69,70,False,yes
9,rainy,75,80,False,yes


In [2]:
def rationalize(list_df) :
    max_df = list_df.max()
    result = list(map(lambda x: x/(max_df)*1, list_df))
    return result

df['temperature'] = rationalize(df['temperature'])
df['humidity'] = rationalize(df['humidity'])
df

,outlook,temperature,humidity,windy,play
0,sunny,1.000000,0.885417,False,no
1,sunny,0.941176,0.937500,True,no
2,overcast,0.976471,0.895833,False,yes
3,rainy,0.823529,1.000000,False,yes
4,rainy,0.800000,0.833333,False,yes
5,rainy,0.764706,0.729167,True,no
6,overcast,0.752941,0.677083,True,yes
7,sunny,0.847059,0.989583,False,no
8,sunny,0.811765,0.729167,False,yes
9,rainy,0.882353,0.833333,False,yes


In [3]:
df['outlook'] = pd.Categorical(df['outlook'])
dfOutlook = pd.get_dummies(df['outlook'], prefix = 'outlook')
dfOutlook

,outlook_overcast,outlook_rainy,outlook_sunny
0,0,0,1
1,0,0,1
2,1,0,0
3,0,1,0
4,0,1,0
5,0,1,0
6,1,0,0
7,0,0,1
8,0,0,1
9,0,1,0


In [4]:
df['windy'] = pd.Categorical(df['windy'])
dfWindy = pd.get_dummies(df['windy'], prefix = 'windy')
dfWindy

,windy_False,windy_True
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0
5,0,1
6,0,1
7,1,0
8,1,0
9,1,0


In [5]:
df['play'] = df['play'].astype('category')
df['play'] = df['play'].cat.codes

df = pd.concat([df, dfOutlook, dfWindy], axis=1)
df = df.drop(columns=['outlook', 'windy'], axis=1)
df

,temperature,humidity,play,outlook_overcast,outlook_rainy,outlook_sunny,windy_False,windy_True
0,1.000000,0.885417,0,0,0,1,1,0
1,0.941176,0.937500,0,0,0,1,0,1
2,0.976471,0.895833,1,1,0,0,1,0
3,0.823529,1.000000,1,0,1,0,1,0
4,0.800000,0.833333,1,0,1,0,1,0
5,0.764706,0.729167,0,0,1,0,0,1
6,0.752941,0.677083,1,1,0,0,0,1
7,0.847059,0.989583,0,0,0,1,1,0
8,0.811765,0.729167,1,0,0,1,1,0
9,0.882353,0.833333,1,0,1,0,1,0


In [6]:
features = df.drop(columns=['play'], axis=1)
targets = df['play']

print (features.head)
print ()
print (targets.head)

<bound method NDFrame.head of     temperature  humidity  outlook_overcast  outlook_rainy  outlook_sunny  \
0      1.000000  0.885417                 0              0              1   
1      0.941176  0.937500                 0              0              1   
2      0.976471  0.895833                 1              0              0   
3      0.823529  1.000000                 0              1              0   
4      0.800000  0.833333                 0              1              0   
5      0.764706  0.729167                 0              1              0   
6      0.752941  0.677083                 1              0              0   
7      0.847059  0.989583                 0              0              1   
8      0.811765  0.729167                 0              0              1   
9      0.882353  0.833333                 0              1              0   
10     0.882353  0.729167                 0              0              1   
11     0.847059  0.937500                 1   

## Training and Testing

In [7]:
features_train, features_test, targets_train, targets_test = train_test_split(features, targets, test_size=0.5, stratify=targets, random_state=None)
print (features_train)
print ()
print (targets_train)
print ()

print (features_test)
print ()
print (targets_test)
print ()

    temperature  humidity  outlook_overcast  outlook_rainy  outlook_sunny  \
1      0.941176  0.937500                 0              0              1   
4      0.800000  0.833333                 0              1              0   
11     0.847059  0.937500                 1              0              0   
9      0.882353  0.833333                 0              1              0   
0      1.000000  0.885417                 0              0              1   
10     0.882353  0.729167                 0              0              1   
2      0.976471  0.895833                 1              0              0   

    windy_False  windy_True  
1             0           1  
4             1           0  
11            0           1  
9             1           0  
0             1           0  
10            0           1  
2             1           0  

1     0
4     1
11    1
9     1
0     0
10    1
2     1
Name: play, dtype: int8

    temperature  humidity  outlook_overcast  outlook_rainy  o

In [8]:
model = MiniBatch(2, 3, 2, 0.1, 0.5, 4)
model.fit(features_train, targets_train)
predict = model.predict(features_test)

print ('Accuracy\t', accuracy_score(targets_test, predict))

[[[0.7099173022265501, 0.3101398077927209], [0.2151664790233605, 0.9693035885086444], [0.5507879462422167, 0.7813355869418027], [0.5451346081820226, 0.7831034448433154], [0.2935068738152382, 0.842360644761262], [0.8954827421871127, 0.6352548297287995], [0.07563971793571045, 0.6607121942627732], [0.6322052947989139, 0.6215837427214219]], [[0.968856450355803, 0.13745402580657506], [0.007573588690421129, 0.187488821955329], [0.35015696127499385, 0.20793512683129423]], [[0.6904448340762837, 0.2037388856014829], [0.21644921068016953, 0.14017093668296932], [0.06802380961633314, 0.9695895910547943]], [[0.3549883072657679], [0.2535626920614952], [0.10361163967848919]]]

[[[0.1863468832724118, 0.21610174416750685], [0.1917846448445406, 0.4835616766001054], [0.16638835411124736, 0.24075668443342804], [0.05984499453984769]]]

[[[0.8946745796644076, 0.9490016067288152], [0.8577388335628816, 0.605791626917487], [0.8143025672989037, 0.6242737042877367], [0.7095911721698079]]]
0
0
0
0
ganti iterasi
0

IndexError: list index out of range